## Introducción al Machine Learning
### Clustering

Este cuaderno muestra un ejemplo de como calcular el coeficiente Silhouette para varios numeros 
de cluster y así elegir un número adecuado. Además se muestra como hacer un gráfico de Silhouette

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
import pickle

Pickle con las series históricas de los componentes del IBEX35

In [ ]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
close_dict = {ticker: df.close for ticker, df in stock_data.items()} 
close_data = pd.DataFrame(close_dict) 

Haremos clustering de las empresas del IBEX tomando 
como características la matriz de correlación de los rendimientos

In [ ]:
close_df = close_data['2018-01-02':]

In [ ]:
close_df = close_df.dropna(axis=1)
close_df

In [ ]:
stock_ret = np.log(close_df).diff()[1:]
stock_ret.head()

In [ ]:
corr_matrix = stock_ret.corr()
corr_matrix

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
features = corr_matrix

Un bucle en el que cambiamos el número de clusters.  Hacemos el clustering y almacenamos el coeficiente de Silhouette

In [ ]:
silhouette_avgs = dict()
for i_clusters in range(2, 15):
    kmeans = KMeans(n_clusters=i_clusters, random_state=10)
    cluster_labels = kmeans.fit_predict(features)
    silhouette_avgs[i_clusters] = silhouette_score(features, cluster_labels)

In [ ]:
silh_by_clusters = pd.Series(silhouette_avgs)
silh_by_clusters

In [ ]:
silh_by_clusters.plot()

#### El gráfico de Silhouette

In [ ]:
n_clusters = 10

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,8))

# X Valores negativos bajos hasta 1.0
# Y Proporcional al numero de clusters y ejemplos
ax1.set_xlim([-0.1, 1])
ax1.set_ylim([0, features.shape[0] + (n_clusters + 1) * 10])

kmeans = KMeans(n_clusters=n_clusters, random_state=10)
cluster_labels = kmeans.fit_predict(features)

stock_labels = pd.Series(cluster_labels, index=features.columns)

# El dictionario con los grupos
for i in range(n_clusters):
        print("Cluster ",i,":",list(stock_labels[stock_labels == i].index))

silhouette_avg = silhouette_score(features, cluster_labels)

print("silhouette:", silhouette_avg)
print("")

# silhouette scores de cada ejemplo
silhouette_values = silhouette_samples(features, cluster_labels)

y_lower = 10

for i in range(n_clusters):
    # agrupar los silhouette scores del cluster y ordenar
    i_cluster_sil_values = silhouette_values[cluster_labels == i]
    i_cluster_sil_values.sort()

    size_cluster_i = i_cluster_sil_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / n_clusters)
    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                      0, i_cluster_sil_values,
                      facecolor=color, edgecolor=color, alpha=0.7)

    # Label the silhouette plots with their cluster numbers at the middle
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

ax1.set_title("Gráfico Silhouette")
ax1.set_xlabel("Coeficiente Silhouette")
ax1.set_ylabel("Cluster")

# vertical donde cae el promedio 
ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

ax1.set_yticks([]) 
plt.show()

____

### Ejercicio Propuesto

1. Probar el gráfico Silhouette para distinto número de clusters
2. Probar el clustering utilizando las rentabilidades mensuales en lugar de la matriz de correlación